# Modelling

In [1]:
import pandas as pd
import numpy as np
import re
import time
import os

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Perceptron, LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer

from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline

rng = np.random.RandomState(42)

# from keras import Sequential
# from keras.layers import Dense

import warnings
warnings.filterwarnings('ignore') # hide warnings to avoid cluttering the notebook output

In [2]:
class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None
        self._elapsed_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._elapsed_time = None
        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        self._elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        print(f"Elapsed time: {self._elapsed_time:0.4f} seconds")

    def duration(self):
        """Return the elapsed time from the timer."""
        if self._elapsed_time is None:
            raise TimerError(f"Timer has not run. Use .start() and .stop() to start and stop the timer.")

        return self._elapsed_time

In [3]:
df = pd.read_csv('../assets/df_merge_final.csv')

In [4]:
def create_dt_features(dataframe):
    dataframe['daylight_duration'] = pd.to_timedelta(dataframe['daylight_duration'])
    dataframe['Daylight_hours'] = dataframe['daylight_duration'].dt.total_seconds() / (60*60)
    dataframe['Date'] = pd.to_datetime(dataframe['Date'])
    dataframe['Month'] = dataframe['Date'].dt.month
    dataframe['Day'] = dataframe['Date'].dt.day
    return dataframe

In [5]:
df = create_dt_features(df)

In [6]:
# specify features
num_features = ['Latitude', 'Longitude', 'AddressAccuracy', 'Tavg', 'Depart', 'Heat', 'PrecipTotal', 'SeaLevel', 'ResultDir', 'AvgSpeed', 'Humidity', 'Daylight_hours', 'Month', 'Day']
cat_features = ['Species', 'Trap']
text_features = ['CodeSum']

features = num_features + cat_features + text_features

# define metric to optimise during GridSearch
score_metric = 'roc_auc'

In [7]:
X = df[features]
y = df['WnvPresent']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=rng, stratify=y)

In [9]:
# Processing steps before modelling (standardisation, one hot encoding, over/under-sampling)
minmax = MinMaxScaler(feature_range=(0, 1))
ohe = OneHotEncoder(drop='if_binary', handle_unknown='ignore')
count = CountVectorizer()

sample_smotetomek = SMOTETomek(random_state=rng, n_jobs=2)

text_pipe = Pipeline([
    ("squeeze", FunctionTransformer(lambda x: x.squeeze())),
    ('vector', count),
    ('array', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
    ('scale', minmax)
])

col_transform = ColumnTransformer([
    ('MinMaxScaler', minmax, num_features),
    ('OneHotEncoder', ohe, cat_features),
    ('CountVectorizer', text_pipe, text_features)
], remainder='drop'
)

In [10]:
# WORKING FILE
# perceptron = Perceptron(random_state=42)

# model = Sequential()
# model.add(Dense(50, input_dim=53, activation='relu')) # input
# model.add(Dense(100, activation='relu')) # hidden
# model.add(Dense(50, activation='relu'))  # hidden
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['binary_crossentropy']) 


In [11]:
# Instantiating models for classification
lr = LogisticRegression(random_state=rng)
knn = KNeighborsClassifier()
pct = Perceptron(random_state=rng)
rf = RandomForestClassifier(random_state=rng)
ada = AdaBoostClassifier(random_state=rng)
gbc = GradientBoostingClassifier(random_state=rng)

In [12]:
# create dict to store model performance results for comparison
model_dict = {
    'model_list': [],
    'mean_train_score': [],
    'mean_test_score': [],
    'best_params': [],
    'runtime': []
}

In [13]:
# run pipe, fit params and return best estimator
def run_pipe(clf, pipe_params):
    t = Timer()
    model_name = re.match(r'^(\w+)(?=\()', str(clf)).group()
    print(f"Fitting {model_name}")

    # create pipe
    pipe = Pipeline([
        ('transform', col_transform),
        ('sample', sample_smotetomek),
        ('clf', clf)
    ])
    
    
    # create GridSearchCV
    grid = GridSearchCV(
        estimator = pipe,
        param_grid = pipe_params,
        scoring = score_metric,
        n_jobs = -3,
        cv = 3,
        verbose = 1,
        error_score='raise'
    )

    t.start()
    grid.fit(X_train, y_train)
    t.stop()
    
    test_score = cross_val_score(grid.best_estimator_, X_test, y_test, scoring=score_metric, cv=3, n_jobs=-3)
    # store average scores
    model_dict['model_list'].append(model_name)
    model_dict['mean_train_score'].append(grid.best_score_)
    model_dict['mean_test_score'].append(test_score.mean())
    model_dict['best_params'].append(grid.best_params_)
    model_dict['runtime'].append(t.duration())

    print("Best Score: ", grid.best_score_)
    print("Best Params: ", grid.best_params_)
    print()
    return grid.best_estimator_

In [14]:
# create table with model performance results
def table_model_results():
    model_performance = pd.DataFrame(model_dict)
    
    model_performance.insert(
        loc = 3,
        column = 'score_delta',
        value = abs(model_performance['mean_train_score'] - model_performance['mean_test_score'])
        )

    return model_performance.sort_values(by="mean_train_score", ascending=False).round({'mean_train_score': 4, 'mean_test_score':4, 'score_delta':4, 'runtime': 2})

In [15]:
lr_params = {
    'clf__solver': ['liblinear', 'newton-cg', 'lbfgs'],
    'clf__penalty': ['l2'],
    'clf__C': [100, 10, 1.0, 0.1, 0.01],
    'clf__max_iter': [100, 150]
}

knn_params = {
    'clf__n_neighbors': [3, 5, 9, 15],
    'clf__weights': ['uniform', 'distance']
}

pct_params = {
    'clf__penalty': [None, 'l1', 'l2', 'elasticnet'],
    'clf__class_weight': [None, 'balanced']
}

rf_params = {
    'clf__ccp_alpha': [0.0, 0.01, 0.1],
    'clf__max_features': ['sqrt', 'log2'],
    'clf__n_estimators': [700], #[500, 700, 900]
}

ada_params = {
    'clf__learning_rate': [1.0, 2.0, 10],
    'clf__n_estimators': [200], #[150, 200, 250]
}

gbc_params = {
    'clf__learning_rate': [0.001, 0.01, 0.1],
    'clf__subsample': [0.5, 0.7, 1.0],
    'clf__max_depth': [3, 7, 9],
    'clf__n_estimators': [250, 300, 350]
}

In [16]:
lr_best = run_pipe(lr, lr_params)

Fitting LogisticRegression
Fitting 3 folds for each of 30 candidates, totalling 90 fits
Elapsed time: 115.7490 seconds
Best Score:  0.7970909840569506
Best Params:  {'clf__C': 10, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'newton-cg'}



In [17]:
knn_best = run_pipe(knn, knn_params)

Fitting KNeighborsClassifier
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Elapsed time: 42.8334 seconds
Best Score:  0.7043010030542578
Best Params:  {'clf__n_neighbors': 15, 'clf__weights': 'uniform'}



In [18]:
pct_best = run_pipe(pct, pct_params)

Fitting Perceptron
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Elapsed time: 36.0741 seconds
Best Score:  0.7820190367853201
Best Params:  {'clf__class_weight': None, 'clf__penalty': None}



In [19]:
rf_best = run_pipe(rf, rf_params)

Fitting RandomForestClassifier
Fitting 3 folds for each of 18 candidates, totalling 54 fits
Elapsed time: 442.4903 seconds
Best Score:  0.8053398107785551
Best Params:  {'clf__ccp_alpha': 0.01, 'clf__max_features': 'sqrt', 'clf__n_estimators': 700}



In [20]:
ada_best = run_pipe(ada, ada_params)

Fitting AdaBoostClassifier
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Elapsed time: 30.9919 seconds
Best Score:  0.8157495147485992
Best Params:  {'clf__learning_rate': 1.0, 'clf__n_estimators': 200}



In [21]:
gbc_best = run_pipe(gbc, gbc_params)

Fitting GradientBoostingClassifier
Fitting 3 folds for each of 81 candidates, totalling 243 fits


KeyboardInterrupt: 

In [ ]:
table_model_results()

In [ ]:
# check coefficents from LogisticRegression
# feature_names = lr_best[0].get_feature_names_out()
# coefficients = lr_best[2].coef_
# features_coefs = pd.DataFrame([feature_names, coefficients.reshape(52,)]).transpose()
# features_coefs.rename(columns={0: 'feature', 1: 'coef'}, inplace=True)
# features_coefs.sort_values(by='coef', ascending=False)

In [ ]:
# this should stop the kernel

In [ ]:
# create VotingClassifier ensemble with top performing classifiers and new params
# voting_clf = VotingClassifier([
#     ('GradientBoost', GradientBoostingClassifier(n_estimators=300, random_state=rng)),
#     ('AdaBoost', AdaBoostClassifier(n_estimators=200, random_state=rng)),
#     ('RandomForest', RandomForestClassifier(n_estimators=700, max_features='sqrt', random_state=rng))
# ], voting='soft')

# voting_params = {
#     'sample__sampling_strategy': ['auto', 0.1, 0.3],
#     'clf__GradientBoost__learning_rate': [0.01, 0.1],
#     'clf__GradientBoost__subsample': [0.7, 1.0],
#     'clf__GradientBoost__max_depth': [3, 7],
#     'clf__AdaBoost__learning_rate': [1.0, 2.0, 10],
#     'clf__LogReg__solver': ['liblinear'],
#     'clf__LogReg__C': [100, 10, 1.0],
}

In [ ]:
# create VotingClassifier ensemble with best performing classifiers
voting_clf = VotingClassifier([
    ('GradientBoost', gbc_best[-1]),
    ('AdaBoost', ada_best[-1]),
    ('RandomForest', rf_best[-1])
], voting='soft')

voting_params = {
    'sample__sampling_strategy': ['auto', 0.1, 0.25, 0.5],
#     'clf__GradientBoost__learning_rate': [0.01, 0.1],
#     'clf__GradientBoost__subsample': [0.7, 1.0],
#     'clf__GradientBoost__max_depth': [3, 7],
#     'clf__AdaBoost__learning_rate': [1.0, 2.0, 10],
#     'clf__LogReg__solver': ['liblinear'],
#     'clf__LogReg__C': [100, 10, 1.0],
}

In [ ]:
voting_best = run_pipe(voting_clf, voting_params)

In [ ]:
table_model_results()

In [ ]:
preds = voting_best.predict(X_test)
cm_disp = ConfusionMatrixDisplay.from_predictions(y_test, preds)

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
# voting_best.fit(X, y)

In [ ]:
df_test = pd.read_csv('../assets/df_merge_test.csv')

In [ ]:
df_test = create_dt_features(df_test)

In [ ]:
results = voting_best.predict(df_test[X.columns])

# results = ada_best.predict(df_test[X.columns])

In [ ]:
def create_predictions(predictions, filename):
    if not os.path.isdir('../output'):
        os.mkdir('../output')
    
    if not os.path.isfile(f'../output/{prediction_filename}.csv'):
        results_df = pd.DataFrame(predictions)
        results_df.insert(0, column='id', value=range(1,116294))
        results_df.to_csv(f"../output/{filename}.csv", header=["Id", "WnvPresent"], index=False)
    else:
        raise NameError(f'{prediction_filename}.csv already exists!')

In [ ]:
prediction_filename = 'predictions_10'

create_predictions(results, prediction_filename)

In [ ]:
table_model_results().to_csv(f'../output/params_{prediction_filename}.csv', index=False)